## Standard Imports

In [2]:
import ee
import os
from pathlib import Path
import requests
from PIL import Image
import numpy as np
from io import BytesIO


print('Imported')

Imported


In [3]:

# Initialize Earth Engine
try:
    ee.Initialize(project = 'glacier-probe-model-475519')
except:
    ee.Authenticate()
    ee.Initialize(project = 'glacier-probe-model-475519')

print('Done')

Done


## 1. Hasdeo Forest RGB Downloader

In [ ]:
"""
Hasdeo Forest GeoTIFF Downloader - MODEL TRAINING VERSION
=========================================================
Downloads high-quality multi-band GeoTIFF satellite imagery from Sentinel-2
for the Hasdeo Arand forest region, covering full deforestation timeline.

BANDS DOWNLOADED:
  - B2 (Blue, 490nm), B3 (Green, 560nm), B4 (Red, 665nm)
  - B8 (NIR, 842nm) — critical for NDVI / vegetation indices
  - B11 (SWIR1, 1610nm), B12 (SWIR2, 2190nm) — for burn/soil detection
  - NDVI (computed) — pre-computed vegetation index
  - SCL (Scene Classification Layer) — for masking

OUTPUT:
  - Full-resolution GeoTIFF (10m or 20m) with all bands
  - Cloud-masked composites (median per season)
  - Individual scene downloads
  - Metadata CSV for dataset bookkeeping

USAGE:
  pip install earthengine-api google-cloud-storage requests pillow numpy tqdm
  python hasdeo_geotiff_downloader.py
"""

import ee
import os
import csv
import time
import json
import math
import requests
from pathlib import Path
from datetime import datetime, timedelta
from tqdm import tqdm

# ─────────────────────────────────────────────
# CONFIGURATION
# ─────────────────────────────────────────────

PROJECT_ID = 'glacier-probe-model-475519'   # ← your GEE project ID
GCS_BUCKET = ''                              # ← your GCS bucket name for large exports (optional)
OUTPUT_DIR = 'hasdeo_training_dataset'

# ── Hasdeo Arand full region + sub-regions of interest ──
REGIONS = {
    # Full Hasdeo Arand forest block (~10km x 10km)
    'Hasdeo_Full':       [82.60, 22.80, 83.00, 23.20],

    # PEKB (Parsa East & Kante Basan) — primary deforestation zone
    'PEKB_Core':         [82.70, 22.90, 82.85, 23.05],

    # Kente Extension block — active 2023-2026
    'Kente_Extension':   [82.65, 23.05, 82.80, 23.20],

    # Buffer / control area (less disturbed forest for comparison)
    'Control_Forest':    [82.85, 23.00, 83.00, 23.15],
}

# ── Deforestation timeline: quarterly sampling for dense coverage ──
# Format: (start_date, end_date, priority_label)
# Higher frequency during rapid expansion phases
TIME_PERIODS = []

def build_time_periods():
    """Build quarterly time windows across the full deforestation timeline."""
    periods = []

    # 2013-2015: Initial clearing — semi-annual
    for year in range(2013, 2016):
        periods.append((f'{year}-01-01', f'{year}-07-01', 'initial_clearing'))
        periods.append((f'{year}-07-01', f'{year+1}-01-01', 'initial_clearing'))

    # 2016-2019: Rapid expansion — quarterly
    for year in range(2016, 2020):
        for q, (m1, m2) in enumerate([('01','04'),('04','07'),('07','10'),('10','12')], 1):
            end_m = '01' if q == 4 else m2
            end_y = year + 1 if q == 4 else year
            periods.append((f'{year}-{m1}-01', f'{end_y}-{end_m}-01', 'rapid_expansion'))

    # 2020-2022: Slowdown — quarterly
    for year in range(2020, 2023):
        for q, (m1, m2) in enumerate([('01','04'),('04','07'),('07','10'),('10','12')], 1):
            end_m = '01' if q == 4 else m2
            end_y = year + 1 if q == 4 else year
            periods.append((f'{year}-{m1}-01', f'{end_y}-{end_m}-01', 'slowdown'))

    # 2023-2025: Renewed surge — monthly for key months
    for year in range(2023, 2026):
        for month in range(1, 13):
            start = f'{year}-{month:02d}-01'
            if month == 12:
                end = f'{year+1}-01-01'
            else:
                end = f'{year}-{month+1:02d}-01'
            if year == 2025 and month > 8:
                break
            periods.append((start, end, 'renewed_surge'))

    return periods

TIME_PERIODS = build_time_periods()

# ── Sentinel-2 bands for model training ──
S2_BANDS = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12', 'SCL']
BAND_LABELS = {
    'B2':  'Blue_490nm',
    'B3':  'Green_560nm',
    'B4':  'Red_665nm',
    'B8':  'NIR_842nm',
    'B8A': 'NIR_narrow_865nm',
    'B11': 'SWIR1_1610nm',
    'B12': 'SWIR2_2190nm',
    'SCL': 'SceneClassification',
}

SCALE_METERS = 10        # 10m native resolution for B2/B3/B4/B8; 20m resampled up
MAX_CLOUD_COVER = 20     # % cloud threshold per scene
MIN_IMAGES_PER_PERIOD = 1
EXPORT_MODE = 'drive'    # 'drive' | 'gcs' | 'local_thumb'
                         # 'drive' → exports to your Google Drive (recommended for large files)
                         # 'gcs'   → exports to GCS bucket (fastest for bulk)
                         # 'local_thumb' → downloads small thumbs locally (no quota needed)

# ─────────────────────────────────────────────
# INITIALIZATION
# ─────────────────────────────────────────────

def init_ee():
    try:
        ee.Initialize(project=PROJECT_ID)
        print(f"✓ Earth Engine initialized | project: {PROJECT_ID}")
    except Exception:
        print("⚠ Re-authenticating...")
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)
        print("✓ Authenticated and initialized.")

def create_dirs():
    for sub in ['exports', 'metadata', 'composites', 'logs']:
        Path(f"{OUTPUT_DIR}/{sub}").mkdir(parents=True, exist_ok=True)
    print(f"✓ Output directories ready: {OUTPUT_DIR}/")

# ─────────────────────────────────────────────
# CLOUD MASKING & PREPROCESSING
# ─────────────────────────────────────────────

def mask_s2_clouds(image):
    """
    Robust cloud masking using:
    1. QA60 bitmask (clouds + cirrus)
    2. SCL layer (cloud shadow, medium/high probability cloud, cirrus)
    """
    qa = image.select('QA60')
    cloud_bit  = 1 << 10
    cirrus_bit = 1 << 11
    qa_mask = qa.bitwiseAnd(cloud_bit).eq(0).And(
               qa.bitwiseAnd(cirrus_bit).eq(0))

    scl = image.select('SCL')
    # SCL classes to EXCLUDE: 3=shadow, 8=cloud_med, 9=cloud_high, 10=cirrus, 11=snow
    scl_mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))

    return image.updateMask(qa_mask.And(scl_mask))

def add_indices(image):
    """Add vegetation and deforestation-relevant indices."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')   # water
    nbr  = image.normalizedDifference(['B8', 'B12']).rename('NBR')   # burn ratio
    evi  = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': image.select('B8'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('EVI')
    return image.addBands([ndvi, ndwi, nbr, evi])

def get_s2_collection(roi, start_date, end_date):
    """Get cloud-filtered, index-enhanced Sentinel-2 collection."""
    col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(roi)
           .filterDate(start_date, end_date)
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))
           .select(S2_BANDS + ['QA60'])
           .map(mask_s2_clouds)
           .map(add_indices))
    return col

# ─────────────────────────────────────────────
# EXPORT FUNCTIONS
# ─────────────────────────────────────────────

def export_to_drive(image, description, region, scale=SCALE_METERS):
    """
    Export full-resolution GeoTIFF to Google Drive.
    Best for model training — preserves geospatial metadata.
    """
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder='Hasdeo_Training_Dataset',
        fileNamePrefix=description,
        region=region,
        scale=scale,
        crs='EPSG:32644',           # UTM Zone 44N — appropriate for Hasdeo
        fileFormat='GeoTIFF',
        maxPixels=1e10,
        formatOptions={
            'cloudOptimized': True  # Cloud-Optimized GeoTIFF for fast ML reads
        }
    )
    task.start()
    return task

def export_to_gcs(image, description, region, bucket, scale=SCALE_METERS):
    """Export to Google Cloud Storage bucket."""
    task = ee.batch.Export.image.toCloudStorage(
        image=image,
        description=description,
        bucket=bucket,
        fileNamePrefix=f'hasdeo/{description}',
        region=region,
        scale=scale,
        crs='EPSG:32644',
        fileFormat='GeoTIFF',
        maxPixels=1e10,
        formatOptions={'cloudOptimized': True}
    )
    task.start()
    return task

def download_local_thumb(image, region, filepath, bands=['B4','B3','B2'], size=512):
    """
    Download small PNG thumbnail locally (no Drive/GCS needed).
    Use for quick visual QA only — not for model training.
    """
    try:
        vis = image.visualize(bands=bands, min=0, max=3000)
        url = vis.getThumbURL({'region': region, 'dimensions': size, 'format': 'png'})
        r = requests.get(url, timeout=120)
        if r.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(r.content)
            return True
    except Exception as e:
        print(f"  ✗ Thumb error: {str(e)[:60]}")
    return False

# ─────────────────────────────────────────────
# COMPOSITE GENERATION (for training labels)
# ─────────────────────────────────────────────

def create_seasonal_composite(roi, year, season='dry'):
    """
    Create seasonal median composite — excellent for change detection training.
    dry   = Nov-Apr (clearer skies, best for deforestation detection)
    wet   = May-Oct
    """
    if season == 'dry':
        start = f'{year}-11-01'
        end   = f'{year+1}-04-30'
    else:
        start = f'{year}-05-01'
        end   = f'{year}-10-31'

    col = get_s2_collection(roi, start, end)
    composite = col.median().clip(roi)
    return composite, col.size().getInfo()

def create_annual_mosaic(roi, year):
    """Best-pixel annual mosaic using minimum cloud score."""
    col = get_s2_collection(roi, f'{year}-01-01', f'{year}-12-31')
    # Use median as best-pixel approximation
    mosaic = col.median().clip(roi)
    return mosaic, col.size().getInfo()

# ─────────────────────────────────────────────
# METADATA TRACKING
# ─────────────────────────────────────────────

metadata_records = []

def log_export(task_id, description, region_name, start_date, end_date,
               image_count, phase, export_type):
    metadata_records.append({
        'task_id':      task_id,
        'description':  description,
        'region':       region_name,
        'start_date':   start_date,
        'end_date':     end_date,
        'image_count':  image_count,
        'phase':        phase,
        'export_type':  export_type,
        'timestamp':    datetime.now().isoformat(),
        'scale_m':      SCALE_METERS,
        'crs':          'EPSG:32644',
        'bands':        ','.join(S2_BANDS + ['NDVI','NDWI','NBR','EVI']),
    })

def save_metadata_csv():
    if not metadata_records:
        return
    csv_path = f"{OUTPUT_DIR}/metadata/export_log.csv"
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=metadata_records[0].keys())
        writer.writeheader()
        writer.writerows(metadata_records)
    print(f"✓ Metadata saved: {csv_path}")

def save_task_ids(tasks):
    """Save all GEE task IDs so you can monitor them."""
    task_file = f"{OUTPUT_DIR}/logs/task_ids.json"
    task_data = [{'id': t.id, 'description': t.config['description'],
                  'status': t.status()['state']} for t in tasks if t]
    with open(task_file, 'w') as f:
        json.dump(task_data, f, indent=2)
    print(f"✓ Task IDs saved: {task_file}")
    print(f"  Monitor at: https://code.earthengine.google.com/tasks")

# ─────────────────────────────────────────────
# MAIN EXECUTION
# ─────────────────────────────────────────────

def main():
    print("=" * 70)
    print("HASDEO FOREST — GeoTIFF DOWNLOADER FOR MODEL TRAINING")
    print("=" * 70)
    print(f"Export mode : {EXPORT_MODE.upper()}")
    print(f"Resolution  : {SCALE_METERS}m")
    print(f"Cloud cover : <{MAX_CLOUD_COVER}%")
    print(f"Time periods: {len(TIME_PERIODS)}")
    print(f"Regions     : {list(REGIONS.keys())}")
    print(f"Bands       : {S2_BANDS} + NDVI, NDWI, NBR, EVI")
    print("=" * 70)

    init_ee()
    create_dirs()

    all_tasks = []
    export_count = 0
    skipped_count = 0

    # ── PART 1: Individual scene / period exports ──────────────────────────
    print("\n[ PART 1 ] Exporting per-period composites...")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n🌳 Region: {region_name}")

        for start_date, end_date, phase in tqdm(TIME_PERIODS, desc=f"  {region_name}"):
            try:
                col = get_s2_collection(roi, start_date, end_date)
                count = col.size().getInfo()

                if count < MIN_IMAGES_PER_PERIOD:
                    skipped_count += 1
                    continue

                # Use median composite for the period
                composite = col.median().clip(roi)

                # Safe description (GEE has 100-char limit, no special chars)
                desc = (f"{region_name}_{start_date[:7]}_{phase[:8]}"
                        .replace('-', '_').replace(' ', '_'))[:90]

                if EXPORT_MODE == 'drive':
                    task = export_to_drive(composite, desc, roi)
                    all_tasks.append(task)
                    export_count += 1
                    print(f"    ✓ Queued: {desc} ({count} scenes)")

                elif EXPORT_MODE == 'gcs':
                    if not GCS_BUCKET:
                        raise ValueError("Set GCS_BUCKET in config for gcs mode")
                    task = export_to_gcs(composite, desc, roi, GCS_BUCKET)
                    all_tasks.append(task)
                    export_count += 1

                elif EXPORT_MODE == 'local_thumb':
                    # Quick local download (PNG thumbs for visual QA)
                    thumb_path = f"{OUTPUT_DIR}/exports/{desc}.png"
                    if not os.path.exists(thumb_path):
                        success = download_local_thumb(composite, roi, thumb_path)
                        if success:
                            export_count += 1
                    else:
                        skipped_count += 1

                log_export(
                    task_id=all_tasks[-1].id if all_tasks and EXPORT_MODE != 'local_thumb' else 'local',
                    description=desc,
                    region_name=region_name,
                    start_date=start_date,
                    end_date=end_date,
                    image_count=count,
                    phase=phase,
                    export_type=EXPORT_MODE
                )

                time.sleep(0.5)  # Rate limit GEE API calls

            except Exception as e:
                print(f"    ✗ Error [{start_date}]: {str(e)[:60]}")
                continue

    # ── PART 2: Annual mosaics (2013-2025) ─────────────────────────────────
    print("\n\n[ PART 2 ] Exporting annual mosaics (best for change detection)...")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        print(f"\n🌳 Region: {region_name} — Annual Mosaics")

        for year in range(2013, 2026):
            try:
                mosaic, count = create_annual_mosaic(roi, year)
                if count == 0:
                    print(f"  {year}: No data")
                    continue

                desc = f"ANNUAL_{region_name}_{year}"[:90]

                if EXPORT_MODE == 'drive':
                    task = export_to_drive(mosaic, desc, roi)
                    all_tasks.append(task)
                    export_count += 1
                    print(f"  ✓ {year}: {count} scenes → {desc}")

                elif EXPORT_MODE == 'gcs':
                    task = export_to_gcs(mosaic, desc, roi, GCS_BUCKET)
                    all_tasks.append(task)
                    export_count += 1

                elif EXPORT_MODE == 'local_thumb':
                    thumb_path = f"{OUTPUT_DIR}/composites/{desc}.png"
                    download_local_thumb(mosaic, roi, thumb_path)
                    export_count += 1

                log_export(
                    task_id=all_tasks[-1].id if all_tasks and EXPORT_MODE != 'local_thumb' else 'local',
                    description=desc,
                    region_name=region_name,
                    start_date=f'{year}-01-01',
                    end_date=f'{year}-12-31',
                    image_count=count,
                    phase='annual_mosaic',
                    export_type=EXPORT_MODE
                )

                time.sleep(0.5)

            except Exception as e:
                print(f"  ✗ {year}: {str(e)[:60]}")

    # ── PART 3: Seasonal dry-season composites ──────────────────────────────
    print("\n\n[ PART 3 ] Dry-season composites (Nov-Apr, clearest images)...")
    print("-" * 70)

    for region_name, coords in REGIONS.items():
        roi = ee.Geometry.Rectangle(coords)
        for year in range(2013, 2025):
            try:
                composite, count = create_seasonal_composite(roi, year, 'dry')
                if count == 0:
                    continue

                desc = f"DRY_{region_name}_{year}_{year+1}"[:90]

                if EXPORT_MODE == 'drive':
                    task = export_to_drive(composite, desc, roi)
                    all_tasks.append(task)
                    export_count += 1
                    print(f"  ✓ Dry {year}-{year+1}: {count} scenes")

                elif EXPORT_MODE == 'gcs':
                    task = export_to_gcs(composite, desc, roi, GCS_BUCKET)
                    all_tasks.append(task)
                    export_count += 1

                log_export(
                    task_id=all_tasks[-1].id if all_tasks and EXPORT_MODE != 'local_thumb' else 'local',
                    description=desc,
                    region_name=region_name,
                    start_date=f'{year}-11-01',
                    end_date=f'{year+1}-04-30',
                    image_count=count,
                    phase='dry_season_composite',
                    export_type=EXPORT_MODE
                )

                time.sleep(0.5)

            except Exception as e:
                print(f"  ✗ Dry {year}: {str(e)[:50]}")

    # ── SUMMARY ──────────────────────────────────────────────────────────────
    print("\n" + "=" * 70)
    print("EXPORT SUMMARY")
    print("=" * 70)
    print(f"✓ Exports queued : {export_count}")
    print(f"⊙ Skipped        : {skipped_count} (no data in period)")
    print(f"📊 Total periods  : {len(TIME_PERIODS)} × {len(REGIONS)} regions")
    print()

    if EXPORT_MODE == 'drive':
        print("📁 Files will appear in Google Drive → 'Hasdeo_Training_Dataset' folder")
        print("⏱  Large exports may take 10 min – several hours depending on size")
        print("🔗 Monitor tasks at: https://code.earthengine.google.com/tasks")
        save_task_ids(all_tasks)
    elif EXPORT_MODE == 'gcs':
        print(f"📁 Files uploading to gs://{GCS_BUCKET}/hasdeo/")
        save_task_ids(all_tasks)
    else:
        print(f"📁 PNG thumbnails saved to: {OUTPUT_DIR}/exports/")

    save_metadata_csv()

    print()
    print("DATASET STRUCTURE:")
    print(f"  • Per-period composites : ~{len(TIME_PERIODS) * len(REGIONS)} images")
    print(f"  • Annual mosaics        : ~{13 * len(REGIONS)} images (2013-2025)")
    print(f"  • Dry-season composites : ~{12 * len(REGIONS)} images")
    print(f"  • Bands per image       : B2,B3,B4,B8,B8A,B11,B12,SCL + NDVI,NDWI,NBR,EVI")
    print(f"  • Format                : Cloud-Optimized GeoTIFF (COG), EPSG:32644")
    print("=" * 70)

if __name__ == '__main__':
    main()

In [14]:
"""
Hasdeo Forest RGB Image Downloader - VIEWABLE VERSION
Downloads RGB satellite imagery for Hasdeo Arand in standard viewable format.
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests
from PIL import Image
import numpy as np
from io import BytesIO

# --- CONFIGURATION ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'hasdeo_forest_dataset_rgb'
NUM_IMAGES = 50  
SCALE = 30  # 30m resolution for manageable file sizes
MAX_CLOUD_COVER = 30  # Reduced for better quality images

# Hasdeo Forest Area of Interest
# Coordinates: [lon_min, lat_min, lon_max, lat_max]
HASDEO_REGION = {
    'Hasdeo_Arand_Zoomed': [82.75, 22.95, 82.85, 23.05]
}

# Year ranges for sampling
YEAR_RANGES = [
    ('2018-01-01', '2019-01-01'),
    ('2019-01-01', '2020-01-01'),
    ('2020-01-01', '2021-01-01'),
    ('2021-01-01', '2022-01-01'),
    ('2022-01-01', '2023-01-01'),
    ('2023-01-01', '2024-01-01'),
    ('2024-01-01', '2025-01-01'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection with cloud filtering"""
    # Sentinel-2 Surface Reflectance with cloud masking
    def maskS2clouds(image):
        qa = image.select('QA60')
        # Bits 10 and 11 are clouds and cirrus
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
            qa.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(mask)
    
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
        .map(maskS2clouds)

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        # Sentinel-2 values are 0-10000, we'll use 0-3000 as typical range
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL (this returns a viewable image)
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 1024,  # 1024x1024 pixels
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            # Save directly as PNG
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: Cloud-masked, normalized to 0-255\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

# --- MAIN EXECUTION ---

def main_hasdeo():
    """Main download function for Hasdeo Forest"""
    print("=" * 70)
    print("HASDEO FOREST IMAGE DOWNLOADER - RGB VIEWABLE VERSION")
    print(f"Targeting Central Hasdeo Region: {HASDEO_REGION['Hasdeo_Arand_Zoomed']}")
    print("=" * 70)
    
    create_output_dirs()
    
    downloaded_count = 0
    failed_count = 0
    
    print(f"\nTarget: {NUM_IMAGES} images (Max Cloud Cover: {MAX_CLOUD_COVER}%)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    region_name = 'Hasdeo_Arand_Zoomed'
    coords = HASDEO_REGION[region_name]
    roi = ee.Geometry.Rectangle(coords)

    print(f"🌳 Starting Downloads for: {region_name}")
    print("-" * 70)
    
    # Iterate through each year range
    for start_date, end_date in YEAR_RANGES:
        if downloaded_count >= NUM_IMAGES:
            break
        
        # Get the satellite collection for the year
        collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
        
        if collection is None:
            print(f"  {start_date[:4]}: 0 images found")
            continue
        
        try:
            count = collection.size().getInfo()
            if count == 0:
                print(f"  {start_date[:4]}: 0 images found")
                continue
                
            print(f"  {start_date[:4]}: {count} images available")
            
            # Limit download per year
            images_to_download = min(10, count, NUM_IMAGES - downloaded_count)
            if images_to_download <= 0:
                break
                
            images = collection.limit(images_to_download).toList(images_to_download)
            
            for i in range(images_to_download):
                if downloaded_count >= NUM_IMAGES:
                    break
                    
                image = ee.Image(images.get(i))
                
                # Get date
                date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                
                # Filename construction (PNG format)
                filename = f"{OUTPUT_DIR}/rgb_images/{region_name}_{date_acquired}_RGB.png"
                metadata_filename = f"{OUTPUT_DIR}/metadata/{region_name}_{date_acquired}_metadata.txt"
                
                # Skip if exists
                if os.path.exists(filename):
                    downloaded_count += 1
                    print(f"    ⊙ {date_acquired} (exists)")
                    continue
                
                print(f"    ↓ {date_acquired}...", end=' ')
                
                if download_rgb_image(image, roi, filename):
                    save_metadata(image, metadata_filename, region_name)
                    downloaded_count += 1
                    print(f"✓ ({downloaded_count}/{NUM_IMAGES})")
                else:
                    failed_count += 1
                
                time.sleep(2)
                
        except Exception as e:
            print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
            continue
            
    # Summary
    print("\n" + "=" * 70)
    print("DOWNLOAD COMPLETE")
    print("=" * 70)
    print(f"✓ Downloaded: {downloaded_count} images")
    print(f"✗ Failed: {failed_count}")
    print(f"📁 Location: {OUTPUT_DIR}/rgb_images/")
    print("\n✓ Images are in standard PNG format and viewable!")
    print("You can open them with any image viewer or photo app.")
    print("=" * 70)

if __name__ == "__main__":
    main_hasdeo()

✓ Earth Engine initialized for project: glacier-probe-model-475519
HASDEO FOREST IMAGE DOWNLOADER - RGB VIEWABLE VERSION
Targeting Central Hasdeo Region: [82.75, 22.95, 82.85, 23.05]
✓ Created output directories in hasdeo_forest_dataset_rgb/

Target: 50 images (Max Cloud Cover: 30%)
Time periods: 7
Resolution: 30m | Format: PNG (Viewable RGB)

🌳 Starting Downloads for: Hasdeo_Arand_Zoomed
----------------------------------------------------------------------
  2018: 11 images available
    ↓ 2018-01-21... ✓ (1/50)
    ↓ 2018-01-31... ✓ (2/50)
    ↓ 2018-02-15... ✓ (3/50)
    ↓ 2018-03-07... ✓ (4/50)
    ↓ 2018-03-12... ✓ (5/50)
    ↓ 2018-04-21... ✓ (6/50)
    ↓ 2018-11-07... ✓ (7/50)
    ↓ 2018-11-17... ✓ (8/50)
    ↓ 2018-12-07... ✓ (9/50)
    ↓ 2018-12-22... ✓ (10/50)
  2019: 41 images available
    ↓ 2019-01-01... ✓ (11/50)
    ↓ 2019-01-06... ✓ (12/50)
    ↓ 2019-01-11... ✓ (13/50)
    ↓ 2019-01-16... ✓ (14/50)
    ↓ 2019-01-21... ✓ (15/50)
    ↓ 2019-01-31... ✓ (16/50)
    ↓ 2019

## 2. Kangaroo Island Black Summer Bushfire Case

In [16]:
"""
Kangaroo Island Black Summer Bushfire Image Downloader
Downloads RGB satellite imagery for Kangaroo Island, South Australia
Covering the 2019-20 Black Summer bushfire period and recovery
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'kangaroo_island_black_summer'
NUM_IMAGES = 100  # More images to capture pre-fire, during, and post-fire periods
SCALE = 30  # 30m resolution for manageable file sizes
MAX_CLOUD_COVER = 100  # Allow ALL images including heavy smoke/clouds

# Kangaroo Island Area of Interest - Black Summer Fire Zone
# Coordinates converted from DMS to decimal degrees:
# Latitude: 35°33'41"S to 36°05'12"S = -35.5614 to -36.0867
# Longitude: 136°32'04"E to 138°00'00"E = 136.5344 to 138.0000
KANGAROO_ISLAND_REGION = {
    'Kangaroo_Island_Fire_Zone': [136.5344, -36.0867, 138.0000, -35.5614]
}

# Black Summer bushfires on Kangaroo Island:
# - Started: December 20, 2019 (lightning strikes)
# - Major fires: December 30, 2019 - February 6, 2020
# - Declared safe: February 6, 2020
# Year ranges: 5 years before (2014-2019) and 5 years after (2020-2025)
YEAR_RANGES = [
    # Pre-fire period (5 years before)
    ('2014-01-01', '2014-12-31'),
    ('2015-01-01', '2015-12-31'),
    ('2016-01-01', '2016-12-31'),
    ('2017-01-01', '2017-12-31'),
    ('2018-01-01', '2018-12-31'),
    # Critical fire year
    ('2019-01-01', '2019-12-19'),  # Pre-fire 2019
    ('2019-12-20', '2020-02-06'),  # During fire (Dec 20, 2019 - Feb 6, 2020)
    ('2020-02-07', '2020-12-31'),  # Post-fire recovery 2020
    # Post-fire recovery period (4 more years)
    ('2021-01-01', '2021-12-31'),
    ('2022-01-01', '2022-12-31'),
    ('2023-01-01', '2023-12-31'),
    ('2024-01-01', '2024-12-31'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection WITHOUT cloud masking to preserve smoke/fire effects"""
    # NO CLOUD MASKING - We want to see smoke, clouds, and fire effects!
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        # Sentinel-2 values are 0-10000, we'll use 0-3000 as typical range
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL (this returns a viewable image)
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 2048,  # Higher resolution for large area (2048x2048)
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            # Save directly as PNG
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name, period_label):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Location: Kangaroo Island, South Australia\n")
            f.write(f"Event: Black Summer Bushfires (2019-20)\n")
            f.write(f"Period: {period_label}\n")
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: NO cloud masking - smoke and fire effects preserved\n")
            f.write(f"\nContext: Area burned ~211,500 ha (48% of island)\n")
            f.write(f"Fire Period: Dec 20, 2019 - Feb 6, 2020\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

def get_period_label(start_date, end_date):
    """Get descriptive label for the time period"""
    if '2019-12-20' in start_date and '2020-02-06' in end_date:
        return "DURING FIRE (Dec 20, 2019 - Feb 6, 2020)"
    elif int(start_date[:4]) < 2019 or (start_date[:4] == '2019' and '12-19' in start_date):
        return f"PRE-FIRE ({start_date[:4]})"
    elif int(start_date[:4]) >= 2020:
        years_after = int(start_date[:4]) - 2020
        return f"POST-FIRE RECOVERY (+{years_after} years - {start_date[:4]})"
    return start_date[:4]

# --- MAIN EXECUTION ---

def main_kangaroo_island():
    """Main download function for Kangaroo Island Black Summer"""
    print("=" * 80)
    print("KANGAROO ISLAND BLACK SUMMER BUSHFIRE IMAGE DOWNLOADER")
    print(f"Coordinates: {KANGAROO_ISLAND_REGION['Kangaroo_Island_Fire_Zone']}")
    print("Fire Period: December 20, 2019 - February 6, 2020")
    print("Coverage: 5 years pre-fire (2014-2019) + fire period + 5 years post-fire (2020-2024)")
    print("=" * 80)
    
    create_output_dirs()
    
    downloaded_count = 0
    failed_count = 0
    
    print(f"\nTarget: {NUM_IMAGES} images (Max Cloud Cover: {MAX_CLOUD_COVER}%)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    region_name = 'Kangaroo_Island_Fire_Zone'
    coords = KANGAROO_ISLAND_REGION[region_name]
    roi = ee.Geometry.Rectangle(coords)

    print(f" Starting Downloads for: Kangaroo Island Black Summer")
    print("-" * 80)
    
    # Iterate through each year range
    for start_date, end_date in YEAR_RANGES:
        if downloaded_count >= NUM_IMAGES:
            break
        
        period_label = get_period_label(start_date, end_date)
        
        # Get the satellite collection for the period
        collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
        
        if collection is None:
            print(f"  {period_label}: 0 images found")
            continue
        
        try:
            count = collection.size().getInfo()
            if count == 0:
                print(f"  {period_label}: 0 images found")
                continue
                
            print(f"  {period_label}: {count} images available")
            
            # Adjust number of images per period
            # More images during fire period, fewer for other years
            if 'DURING FIRE' in period_label:
                images_per_period = min(20, count, NUM_IMAGES - downloaded_count)
            else:
                images_per_period = min(8, count, NUM_IMAGES - downloaded_count)
                
            if images_per_period <= 0:
                break
                
            images = collection.limit(images_per_period).toList(images_per_period)
            
            for i in range(images_per_period):
                if downloaded_count >= NUM_IMAGES:
                    break
                    
                image = ee.Image(images.get(i))
                
                # Get date
                date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                
                # Filename construction (PNG format)
                period_prefix = period_label.split('(')[0].strip().replace(' ', '_').replace('-', '')
                filename = f"{OUTPUT_DIR}/rgb_images/KI_{date_acquired}_{period_prefix}_RGB.png"
                metadata_filename = f"{OUTPUT_DIR}/metadata/KI_{date_acquired}_{period_prefix}_metadata.txt"
                
                # Skip if exists
                if os.path.exists(filename):
                    downloaded_count += 1
                    print(f"    ⊙ {date_acquired} (exists)")
                    continue
                
                print(f"    ↓ {date_acquired}...", end=' ')
                
                if download_rgb_image(image, roi, filename):
                    save_metadata(image, metadata_filename, region_name, period_label)
                    downloaded_count += 1
                    print(f"✓ ({downloaded_count}/{NUM_IMAGES})")
                else:
                    failed_count += 1
                
                time.sleep(2)
                
        except Exception as e:
            print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
            continue
            
    # Summary
    print("\n" + "=" * 80)
    print("DOWNLOAD COMPLETE - KANGAROO ISLAND BLACK SUMMER DATASET")
    print("=" * 80)
    print(f"✓ Downloaded: {downloaded_count} images")
    print(f"✗ Failed: {failed_count}")
    print(f" Location: {OUTPUT_DIR}/rgb_images/")
    print(f"\n Dataset Coverage:")
    print(f"   • Pre-fire baseline: 2014-2019 (5 years)")
    print(f"   • Active fire period: Dec 20, 2019 - Feb 6, 2020")
    print(f"   • Post-fire recovery: 2020-2024 (5 years)")
    print(f"   • Area burned: ~211,500 hectares (48% of island)")
    print(f"\n✓ Images are in standard PNG format and viewable!")
    print("You can open them with any image viewer or photo app.")
    print("=" * 80)

if __name__ == "__main__":
    main_kangaroo_island()

✓ Earth Engine initialized for project: glacier-probe-model-475519
KANGAROO ISLAND BLACK SUMMER BUSHFIRE IMAGE DOWNLOADER
Coordinates: [136.5344, -36.0867, 138.0, -35.5614]
Fire Period: December 20, 2019 - February 6, 2020
Coverage: 5 years pre-fire (2014-2019) + fire period + 5 years post-fire (2020-2024)
✓ Created output directories in kangaroo_island_black_summer/

Target: 100 images (Max Cloud Cover: 100%)
Time periods: 12
Resolution: 30m | Format: PNG (Viewable RGB)

🔥 Starting Downloads for: Kangaroo Island Black Summer
--------------------------------------------------------------------------------
  PRE-FIRE (2014): 0 images found
  PRE-FIRE (2015): 0 images found
  PRE-FIRE (2016): 0 images found
  PRE-FIRE (2017): 4 images available
    ↓ 2017-03-08... ✓ (1/100)
    ↓ 2017-07-21... ✓ (2/100)
    ⊙ 2017-07-21 (exists)
    ↓ 2017-12-26... ✓ (4/100)
  PRE-FIRE (2018): 28 images available
    ↓ 2018-02-06... ✓ (5/100)
    ⊙ 2018-02-06 (exists)
    ↓ 2018-02-26... ✓ (7/100)
    ⊙ 

## 3. Greater Sydney Case

In [18]:
"""
Greater Sydney Blue Mountains Fringe Satellite Image Downloader
Downloads RGB satellite imagery for the Blue Mountains/Warragamba Dam Catchment Area
Captures: Bushfire (Black Summer 2019-20), Drought, Regrowth, and Deforestation
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION (MODIFIED for Sydney Blue Mountains Fringe) ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'sydney_blue_mountains_fringe_rgb'
NUM_IMAGES = 75  # 15 images per sub-region
SCALE = 30  # Increased to 30m to ensure successful downloads
MAX_CLOUD_COVER = 100  # Allow all images including smoke/clouds during fires

# Greater Sydney Blue Mountains Fringe - Multiple Sub-Regions
# Multiple smaller regions to ensure visible imagery capture
# Coordinates: [lon_min, lat_min, lon_max, lat_max]
SYDNEY_REGIONS = {
    'Blue_Mts_Katoomba': [150.25, -33.75, 150.35, -33.65],  # Katoomba/Three Sisters area
    'Blue_Mts_Wentworth_Falls': [150.35, -33.75, 150.45, -33.65],  # Wentworth Falls
    'Warragamba_Dam_North': [150.55, -33.95, 150.65, -33.85],  # North of dam
    'Warragamba_Dam_South': [150.55, -34.05, 150.65, -33.95],  # South of dam
    'Penrith_Urban_Edge': [150.65, -33.80, 150.75, -33.70],  # Urban fringe
}

# Date ranges specifically targeting the Black Summer event and recovery/drought periods
YEAR_RANGES = [
    # Pre-fire/drought baseline
    ('2018-01-01', '2019-01-01'), 
    # Peak drought and fire period (Black Summer)
    ('2019-06-01', '2020-06-01'), 
    # Immediate post-fire and start of regrowth
    ('2020-06-01', '2021-06-01'), 
    # Continued recovery and expansion monitoring
    ('2021-06-01', '2022-06-01'),
    ('2022-06-01', '2023-06-01'),
    ('2023-06-01', '2024-06-01'),
    ('2024-06-01', '2025-06-01'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection WITHOUT cloud masking to preserve all effects"""
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL with adjusted dimensions
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 512,  # Reduced for better compatibility
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name, period_label):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Location: Greater Sydney - Blue Mountains Fringe & Warragamba Catchment\n")
            f.write(f"Research Focus: Bushfire, Drought, Regrowth, Deforestation\n")
            f.write(f"Period: {period_label}\n\n")
            
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: NO cloud masking - smoke and fire effects preserved\n")
            
            f.write(f"\n=== RESEARCH CONTEXT ===\n")
            f.write(f"Black Summer Fires: 2019-2020\n")
            f.write(f"Key Indices for Analysis:\n")
            f.write(f"  • NBR (Normalized Burn Ratio): Fire severity\n")
            f.write(f"  • NDVI (Normalized Difference Vegetation Index): Vegetation health/regrowth\n")
            f.write(f"  • EVI (Enhanced Vegetation Index): Drought impact\n")
            f.write(f"  • NDMI (Normalized Difference Moisture Index): Water stress\n")
            f.write(f"  • NDWI (Normalized Difference Water Index): Drought monitoring\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

def get_period_label(start_date, end_date):
    """Get descriptive label for the time period"""
    if '2018' in start_date:
        return "PRE-FIRE BASELINE (2018)"
    elif '2019-06' in start_date and '2020-06' in end_date:
        return "DROUGHT & BLACK SUMMER FIRES (2019-2020)"
    elif '2020-06' in start_date and '2021' in end_date:
        return "IMMEDIATE POST-FIRE RECOVERY (2020-2021)"
    elif '2021' in start_date:
        return f"POST-FIRE RECOVERY & MONITORING ({start_date[:4]}-{end_date[:4]})"
    elif '2022' in start_date:
        return f"REGROWTH & DEFORESTATION MONITORING ({start_date[:4]}-{end_date[:4]})"
    elif '2023' in start_date or '2024' in start_date:
        return f"LONG-TERM RECOVERY ({start_date[:4]}-{end_date[:4]})"
    return f"{start_date[:4]}-{end_date[:4]}"

# --- MAIN EXECUTION ---

def main_sydney():
    """Main download function for Sydney Blue Mountains Fringe - Multiple Regions"""
    print("=" * 80)
    print("GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER")
    print("Multiple Sub-Regions for Complete Coverage")
    print("=" * 80)
    print(f"📍 Regions: {len(SYDNEY_REGIONS)}")
    for name, coords in SYDNEY_REGIONS.items():
        print(f"   • {name}: {coords}")
    print(f" Event: Black Summer Bushfires (2019-2020)")
    print(f" Context: Prolonged drought + catastrophic fire + urban expansion")
    print(f" Research: Bushfire, Drought, Regrowth, Deforestation")
    print("=" * 80)
    
    create_output_dirs()
    
    total_downloaded = 0
    total_failed = 0
    
    print(f"\nTarget: {NUM_IMAGES} images total (~{NUM_IMAGES // len(SYDNEY_REGIONS)} per region)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    images_per_region = NUM_IMAGES // len(SYDNEY_REGIONS)
    
    # Process each sub-region
    for region_name, coords in SYDNEY_REGIONS.items():
        print(f"\n{'='*80}")
        print(f"🌳 Processing Region: {region_name}")
        print(f"📍 Coordinates: {coords}")
        print(f"🎯 Target: {images_per_region} images")
        print("-" * 80)
        
        roi = ee.Geometry.Rectangle(coords)
        downloaded_count = 0
        failed_count = 0
        
        # Iterate through each year range for this region
        for start_date, end_date in YEAR_RANGES:
            if downloaded_count >= images_per_region:
                break
            
            period_label = get_period_label(start_date, end_date)
            
            # Get the satellite collection for the period
            collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
            
            if collection is None:
                print(f"  {period_label}: 0 images found")
                continue
            
            try:
                count = collection.size().getInfo()
                if count == 0:
                    print(f"  {period_label}: 0 images found")
                    continue
                    
                print(f"  {period_label}: {count} images available")
                
                # Images per period
                if 'BLACK SUMMER' in period_label:
                    images_per_period = min(5, count, images_per_region - downloaded_count)
                else:
                    images_per_period = min(3, count, images_per_region - downloaded_count)
                    
                if images_per_period <= 0:
                    break
                    
                images = collection.limit(images_per_period).toList(images_per_period)
                
                for i in range(images_per_period):
                    if downloaded_count >= images_per_region:
                        break
                        
                    image = ee.Image(images.get(i))
                    
                    # Get date
                    date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                    
                    # Filename construction
                    period_code = period_label.split('(')[0].strip().replace(' ', '_').replace('&', '').replace('-', '')
                    filename = f"{OUTPUT_DIR}/rgb_images/{region_name}_{date_acquired}_{period_code}_RGB.png"
                    metadata_filename = f"{OUTPUT_DIR}/metadata/{region_name}_{date_acquired}_{period_code}_meta.txt"
                    
                    # Skip if exists
                    if os.path.exists(filename):
                        downloaded_count += 1
                        print(f"    ⊙ {date_acquired} (exists)")
                        continue
                    
                    print(f"    ↓ {date_acquired}...", end=' ')
                    
                    if download_rgb_image(image, roi, filename):
                        save_metadata(image, metadata_filename, region_name, period_label)
                        downloaded_count += 1
                        print(f"✓ ({downloaded_count}/{images_per_region})")
                    else:
                        failed_count += 1
                    
                    time.sleep(1.5)
                    
            except Exception as e:
                print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
                continue
        
        total_downloaded += downloaded_count
        total_failed += failed_count
        print(f"\n✓ {region_name}: {downloaded_count} downloaded, {failed_count} failed")
    
    # Final Summary
    print("\n" + "=" * 80)
    print("DOWNLOAD COMPLETE - SYDNEY BLUE MOUNTAINS MULTI-REGION DATASET")
    print("=" * 80)
    print(f"✓ Total Downloaded: {total_downloaded}/{NUM_IMAGES} images")
    print(f"✗ Total Failed: {total_failed}")
    print(f" Location: {OUTPUT_DIR}/rgb_images/")
    print(f"\n Dataset Coverage:")
    print(f"   • Pre-fire baseline: 2018")
    print(f"   • Drought & Black Summer fires: 2019-2020")
    print(f"   • Post-fire recovery: 2020-2021")
    print(f"   • Long-term monitoring: 2021-2025")
    print(f"\n Sub-Regions Captured:")
    for name in SYDNEY_REGIONS.keys():
        print(f"   • {name}")
    print(f"\n Recommended Analysis Indices:")
    print(f"   • NBR (Normalized Burn Ratio): B8-B12 / B8+B12")
    print(f"     → Fire severity mapping, burn scar detection")
    print(f"   • NDVI (Vegetation Index): B8-B4 / B8+B4")
    print(f"     → Vegetation health, regrowth monitoring")
    print(f"   • EVI (Enhanced Vegetation): 2.5*(B8-B4) / (B8+6*B4-7.5*B2+1)")
    print(f"     → Drought stress, canopy density")
    print(f"   • NDMI (Moisture Index): B8-B11 / B8+B11")
    print(f"     → Water stress, drought impact")
    print(f"   • NDWI (Water Index): B3-B8 / B3+B8")
    print(f"     → Drought severity, water availability")
    print(f"\n✓ Images are in standard PNG format and viewable!")
    print("All smoke, fire, and atmospheric effects are preserved.")
    print("=" * 80)

if __name__ == "__main__":
    main_sydney()

✓ Earth Engine initialized for project: glacier-probe-model-475519
🇦🇺 GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER
Multiple Sub-Regions for Complete Coverage
📍 Regions: 5
   • Blue_Mts_Katoomba: [150.25, -33.75, 150.35, -33.65]
   • Blue_Mts_Wentworth_Falls: [150.35, -33.75, 150.45, -33.65]
   • Warragamba_Dam_North: [150.55, -33.95, 150.65, -33.85]
   • Warragamba_Dam_South: [150.55, -34.05, 150.65, -33.95]
   • Penrith_Urban_Edge: [150.65, -33.8, 150.75, -33.7]
🔥 Event: Black Summer Bushfires (2019-2020)
🌵 Context: Prolonged drought + catastrophic fire + urban expansion
📊 Research: Bushfire, Drought, Regrowth, Deforestation
✓ Created output directories in sydney_blue_mountains_fringe_rgb/

Target: 75 images total (~15 per region)
Time periods: 7
Resolution: 30m | Format: PNG (Viewable RGB)


🌳 Processing Region: Blue_Mts_Katoomba
📍 Coordinates: [150.25, -33.75, 150.35, -33.65]
🎯 Target: 15 images
-------------------------------------------------------------------------------

## 4. Hyderbad Forest Case

In [12]:
"""
Hyderabad Region ULTRA HIGH QUALITY Image Exporter
Fixed for GeoTIFF compatibility and automated task starting.
"""

import ee
import time

# --- CONFIGURATION ---
PROJECT_ID = 'glacier-probe-model-475519'
DRIVE_FOLDER = 'Hyderabad_Deforestation_UltraHD'
NUM_IMAGES = 50
SCALE = 10 
MAX_CLOUD_COVER = 5 

# Regions
HYDERABAD_REGIONS = {
    'Hyderabad_Central': [78.300, 17.425, 78.350, 17.465],
    'ORR_Northwest': [78.260, 17.505, 78.310, 17.545],
    'ORR_Northeast': [78.530, 17.495, 78.580, 17.535],
    'ORR_Southwest': [78.260, 17.345, 78.310, 17.385],
    'ORR_Southeast': [78.520, 17.335, 78.570, 17.375],
    'Kompally_Corridor': [78.460, 17.535, 78.510, 17.575],
    'Airport_Zone': [78.400, 17.215, 78.450, 17.255],
    'IT_Corridor_West': [78.310, 17.405, 78.360, 17.445],
    'Industrial_West': [78.200, 17.475, 78.250, 17.515],
    'Eastern_Expansion': [78.570, 17.385, 78.620, 17.425],
    'Gandipet_Periphery': [78.260, 17.365, 78.310, 17.405],
    'Southern_Sprawl': [78.490, 17.285, 78.540, 17.325],
}

YEAR_RANGES = [
    ('2018-01-01', '2019-01-01'), ('2019-01-01', '2020-01-01'),
    ('2020-01-01', '2021-01-01'), ('2021-01-01', '2022-01-01'),
    ('2022-01-01', '2023-01-01'), ('2023-01-01', '2024-01-01'),
    ('2024-01-01', '2025-01-01'),
]

# --- INITIALIZATION ---
try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized: {PROJECT_ID}")
except Exception:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)

# --- FUNCTIONS ---
def maskS2clouds(image):
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    return image.updateMask(mask)

def export_to_drive(image, region, region_name, date_str):
    try:
        rgb = image.select(['B4', 'B3', 'B2']).visualize(
            min=[300, 400, 400], max=[2800, 2500, 2200], gamma=[1.2, 1.3, 1.4]
        )
        
        task = ee.batch.Export.image.toDrive(
            image=rgb,
            description=f'{region_name}_{date_str}',
            folder=DRIVE_FOLDER,
            fileNamePrefix=f'{region_name}_{date_str}',
            region=region,
            scale=SCALE,
            crs='EPSG:4326',
            maxPixels=1e13,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        task.start()
        return task.id
    except Exception as e:
        print(f"✗ Error: {e}")
        return None

# --- MAIN ---
def main():
    exported_count = 0
    images_per_region = max(1, NUM_IMAGES // len(HYDERABAD_REGIONS))

    for region_name, coords in HYDERABAD_REGIONS.items():
        if exported_count >= NUM_IMAGES: break
        roi = ee.Geometry.Rectangle(coords)
        region_count = 0

        for start, end in YEAR_RANGES:
            if exported_count >= NUM_IMAGES or region_count >= images_per_region: break
            
            col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                .filterBounds(roi).filterDate(start, end)\
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', MAX_CLOUD_COVER))\
                .map(maskS2clouds)

            if col.size().getInfo() > 0:
                img = ee.Image(col.first())
                date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                tid = export_to_drive(img, roi, region_name, date)
                if tid:
                    exported_count += 1
                    region_count += 1
                    print(f"✓ Queued {region_name} ({date})")
                time.sleep(0.5)

    print(f"\n✅ {exported_count} tasks are now RUNNING in Earth Engine.")
    print(f"Check progress: https://code.earthengine.google.com/tasks")

if __name__ == "__main__":
    main()

✓ Earth Engine initialized: glacier-probe-model-475519
✓ Queued Hyderabad_Central (2018-01-14)
✓ Queued Hyderabad_Central (2019-01-04)
✓ Queued Hyderabad_Central (2020-01-14)
✓ Queued Hyderabad_Central (2021-01-18)
✓ Queued ORR_Northwest (2018-01-14)
✓ Queued ORR_Northwest (2019-01-04)
✓ Queued ORR_Northwest (2020-01-14)
✓ Queued ORR_Northwest (2021-01-18)
✓ Queued ORR_Northeast (2018-01-14)
✓ Queued ORR_Northeast (2019-01-04)
✓ Queued ORR_Northeast (2020-01-14)
✓ Queued ORR_Northeast (2021-01-18)
✓ Queued ORR_Southwest (2018-01-14)
✓ Queued ORR_Southwest (2019-01-04)
✓ Queued ORR_Southwest (2020-01-14)
✓ Queued ORR_Southwest (2021-01-18)
✓ Queued ORR_Southeast (2018-01-14)
✓ Queued ORR_Southeast (2019-01-04)
✓ Queued ORR_Southeast (2020-01-14)
✓ Queued ORR_Southeast (2021-01-18)
✓ Queued Kompally_Corridor (2018-01-14)
✓ Queued Kompally_Corridor (2019-01-04)
✓ Queued Kompally_Corridor (2020-01-14)
✓ Queued Kompally_Corridor (2021-01-18)
✓ Queued Airport_Zone (2018-01-14)
✓ Queued Airpo

KeyboardInterrupt: 

In [ ]:
import os
from PIL import Image
import numpy as np
import tifffile # pip install tifffile

input_dir = 'Hyderabad_Deforestation_UltraHD'
output_dir = 'Hyderabad_JPEGs'

if not os.path.exists(output_dir): os.makedirs(output_dir)

print("Starting conversion to JPEG...")
for f in os.listdir(input_dir):
    if f.endswith('.tif'):
        # Read the GeoTIFF
        img = tifffile.imread(os.path.join(input_dir, f))
        # Ensure it's 8-bit for JPEG
        if img.dtype != np.uint8:
            img = (img / 256).astype(np.uint8)
        
        # Save as JPEG
        out_path = os.path.join(output_dir, f.replace('.tif', '.jpg'))
        Image.fromarray(img).save(out_path, "JPEG", quality=95)
        print(f"Converted: {out_path}")

In [20]:
"""
Hyderabad Forest & Green Cover RGB Image Downloader
Downloads RGB satellite imagery for Hyderabad's forest areas in viewable PNG format.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'hyderabad_forest_dataset_rgb'
NUM_IMAGES = 50  
SCALE = 30  
MAX_CLOUD_COVER = 30  

# Green zones and forested peripheries in Hyderabad
HYDERABAD_REGIONS = {
    'KBR_Park_Central': [78.40, 17.41, 78.43, 17.44],
    'Mrugavani_National_Park': [78.32, 17.34, 78.36, 17.38],
    'Mahavir_Harin_Vansthali': [78.57, 17.31, 78.61, 17.35],
    'Ananthagiri_Hills_Proxy': [77.85, 17.29, 77.90, 17.34], # Outer Hyderabad
}

#
# YEAR_RANGES = [
#     ('2024-12-01', '2025-01-01'), # Early Winter
#     ('2025-01-01', '2025-02-01'), # Peak Winter
#     ('2025-02-01', '2025-03-01'), # Early Spring
#     ('2025-03-01', '2025-03-29'), # Baseline (Immediate Pre-Event)
# ]

# During the event
# During the event
YEAR_RANGES = [
    ('2025-03-29', '2025-04-03'), # Start and end date for the window
]


# YEAR_RANGES = [
#     ('2025-04-03', '2025-05-01'), # Immediate Aftermath
#     ('2025-05-01', '2025-06-01'), # Pre-Monsoon Heat (Dry Vegetation)
#     ('2025-06-01', '2025-07-01'), # Start of Monsoon (Cloud cover may increase)
#     ('2025-07-01', '2025-08-01'), # Peak Monsoon (Check for regrowth/flooding)
# ]
# --- INITIALIZATION ---


try:
    ee.Initialize(project=PROJECT_ID)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)

# --- CORE FUNCTIONS ---

def create_output_dirs():
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(parents=True, exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(parents=True, exist_ok=True)

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    def maskS2clouds(image):
        qa = image.select('QA60')
        mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
        return image.updateMask(mask)
    
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
        .map(maskS2clouds)
    
    return collection

def download_rgb_image(image, region, filename):
    try:
        rgb_vis = image.select(['B4', 'B3', 'B2']).visualize(min=0, max=3000)
        url = rgb_vis.getThumbURL({'region': region, 'dimensions': 1024, 'format': 'png'})
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
    except Exception as e:
        print(f" Error: {e}")
    return False

# --- MAIN EXECUTION ---

def main_hyderabad():
    print(f"🌳 Starting Hyderabad Forest Image Downloader")
    create_output_dirs()
    
    downloaded_total = 0
    
    for region_name, coords in HYDERABAD_REGIONS.items():
        if downloaded_total >= NUM_IMAGES: break
        
        roi = ee.Geometry.Rectangle(coords)
        print(f"\nProcessing Region: {region_name}")

        for start_date, end_date in YEAR_RANGES:
            if downloaded_total >= NUM_IMAGES: break
            
            collection = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
            count = collection.size().getInfo()
            
            if count == 0: continue
            
            # Take the clearest image from the year
            best_image = collection.sort('CLOUDY_PIXEL_PERCENTAGE').first()
            date_str = ee.Date(best_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            
            filename = f"{OUTPUT_DIR}/rgb_images/{region_name}_{date_str}.png"
            
            print(f"  ↓ Downloading {date_str}...", end=" ")
            if download_rgb_image(best_image, roi, filename):
                downloaded_total += 1
                print(f"Done! ({downloaded_total}/{NUM_IMAGES})")
                time.sleep(1) # Rate limiting

    print(f"\n✅ Finished! Images saved to {OUTPUT_DIR}/rgb_images/")

if __name__ == "__main__":
    main_hyderabad()

🌳 Starting Hyderabad Forest Image Downloader

Processing Region: KBR_Park_Central

Processing Region: Mrugavani_National_Park

Processing Region: Mahavir_Harin_Vansthali

Processing Region: Ananthagiri_Hills_Proxy
  ↓ Downloading 2025-03-31... Done! (1/50)

✅ Finished! Images saved to hyderabad_forest_dataset_rgb/rgb_images/
